In [2]:
#urllib is used to downlaod the utils frile from deeplearning.net

import urllib.request
with urllib.request.urlopen("http://deeplearning.net/tutorial/code/utils.py") as url:
    response = url.read()
target = open('utils.py','w')
target.write(response.decode('utf-8'))
target.close()

In [7]:
#libraries
import math
import tensorflow as tf
import numpy as np
from PIL import Image
from tensorflow.examples.tutorials.mnist import input_data
from utils import tile_raster_images


C:\Users\jssrv\_programs\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\jssrv\_programs\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\jssrv\_programs\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\jssrv\_programs\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\jssrv\_programs\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed

In [8]:
#class that defines the behavious of the RBM
class RBM(object):
    def __init__(self,input_size, output_size):
        #defineing the hyperparameters
        self._input_size = input_size
        self._output_size = output_size
        self.epochs = 5 #epoch is a training iteration
        self.learning_rate = 1.0 #the step used in the gradiate descent
        self.batchsize = 100 #size of how much data will be used for training per sub iteration
        
        #initializing weights and biases as matrices full of zeroes
        self.w = np.zeros([input_size,output_size],np.float32)
        self.hb = np.zeros([output_size],np.float32) #creates ad ninitializes hidden bias
        self.vb = np.zeros([input_size],np.float32) #creates and initializes the visible bias
        
    #fits the results from the weighted visible layer plus the bias into a sigmoid curve
    def prob_h_given_v(self,visible,w,hb):
        #sigmoid
        return tf.nn.sigmoid(tf.matmul(visible,w)+hb)
    
    #fits the results from the weighted hidden layer plus the bias into a sigmoid curve
    def prob_v_given_h(self,probs):
        #sigmoid
        return tf.nn.sigmoid(tf.matmul(hidden, tf.transpose(w))+vb)
    
    #generates the sample probablility
    def sample_prob(self,probs):
        return tf.nn.relu(tf.sign(probs - tf.random_uniform(tf.shape(probs))))
    
    #training method for the model
    def train(self,x):
        #create the placeholder for the parameters
        _w = tf.placeholder("float",[self._input_size],np.float32)
        _hb = tf.placeholder("float",[self._output_size])
        _vb = tf.placeholder("float",[self._input_size])
        
        prv_w = np.zeros([self._input_size,self._output_size],np.float32) #initialize weights
        prv_hb = np.zeros([self._output_size],np.float32)
        prv_vb = np.zeros([self._input_size],np.float32)
        
        cur_w = np.zeros([self._input_size,self._output_size],np.float32)
        cur_hb = np.zeros([self._output],np.float32)
        cur_vb = np.zeros([self._input],np.float32)
        v0 = tf.placeholder("float32",[None,self._input_size])
        
        #initialize with smaple probabilities
        h0 = self.sample_prob(self.prob_h_given_v(v0,_w,_hb))
        v1 = self.sample_prob(self.prob_v_gien_h(h0,_w,_vb))
        h1 = self.prob_h_given_v(v1,_w,_hb)
        
        #create the gradiates
        positive_grad = tf.matmul(tf.transpose(v0),h0)
        negative_grad = tf.matmul(tf.transpose(v1),h1)
        
        #update learning rates for the layers
        update_w = _w + self.learning_rate*(positive_grad - negative_grad)/tf.to_float(tf.shape(v0)[0])
        update_vb = _vb + self.learning_rate*tf.reduce_mean(v0 - v1,0)
        update_hb = _hb + self.learning_rate*tf.reduce_mean(h0 - h1,0)
        
        ##find error rate
        err = tf.reduce_mean(tf.square(v0-v1))
        
        #training loop
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            #for each epoch
            for epoch in range(self.epochs):
                #for each step/batch
                for start, end in zip(range(0,len(x),self.batchsize),range(self.batchsize,len(x),self.batchsize)):
                    batch = x[start:end]
                    cur_w = sess.run(update_w,feed_dict={v0:batch,_w:prv_w,_hb:prv_hb,_vb:prv_vb})
                    cur_hb = sess.run(update_hb,feed_dict={v0:batch,_w:prv_w,_hb:prv_hb,_vb:prv_vb})
                    cur_vb = sess.run(update_vb,feed_dict={v0:batch,_w:prv_w,_hb:prv_hb,_vb:prv_vb})
                    prv_w=cur_w
                    prv_hb=cur_hb
                    prv_vb=cur_vb
                error = sess.run(err,feed_dict={v0:x,_w:cur_w,_vb:cur_vb, _hb: cur_hb})
                print('Epoch: %d'%epoch,'reconstruction error: %f'% error)
            self.w = prv_w
            self.hb = prv_hb
            self.vb = prv_vb
            
    #create expected output for the DBN
    def rbm_output(self,x):
        input_x = tf.constant(x)
        _w = tf.constant(self.w)
        _hb = tf.constant(self.hb)
        out = tf.nn.sigmoid(tf.matmul(input_x,_w)+_hb)
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            return sess.run(out)
            

In [9]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
trX,trY,teX,teY = mnist.train.images,mnist.train.labels,mnist.test.images,mnist.test.labels

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr